# BASELINE NOTEBOOK 

## Metrics

the following metrics will be used for the model:

`auc`= Approximates the AUC (Area under the curve) of the ROC or PR curves. (ROC / PR)

`Precision` 

`Recall `

`F1`

## general `Baseline` for data train

In [3]:
import os, folium, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio.v3 as iio
import geopandas as gpd
from IPython.display import Image, display
import tensorflow as tf
from tensorflow.keras import layers, callbacks, backend, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_recall_curve, recall_score, roc_auc_score, f1_score, precision_score
from tensorflow.keras.applications import EfficientNetB0, DenseNet121
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import pickle

2023-05-25 00:34:28.158110: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 00:34:28.445993: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 00:34:30.412721: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2023-05-25 00:34:30.412889: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2023-05-25 00:34:30.428772: I tensorflow/c/logging.cc:34] Successfully opened dynamic l

In [4]:
labels= gpd.read_file('labels.geojson')
labels_unique = labels[(labels.Type == 'Negative') | (labels.Type == 'CAFOs') | (labels.Type == 'WWTreatment')
           | (labels.Type == 'Landfills') | (labels.Type == 'RefineriesAndTerminals')
           | (labels.Type == 'ProcessingPlants') | (labels.Type == 'Mines')]

In [5]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels_unique.Type.values.reshape(labels_unique.Type.values.shape[0], 1))
mlb.classes_

array(['CAFOs', 'Landfills', 'Mines', 'Negative', 'ProcessingPlants',
       'RefineriesAndTerminals', 'WWTreatment'], dtype=object)

In [6]:
y_train, y_test= train_test_split(y, train_size=0.1, stratify=y)
def invert_ecoding(encoded_labels, categories):
    categories = ['[unk]']+categories
    return np.take(categories, np.argwhere(encoded_labels == 1.0)[:,1])
y_tra_orig = np.argwhere(y_train == 1.0)[:,1]
y_test_orig = np.argwhere(y_test == 1.0)[:,1]

In [7]:
y_test_orig

array([3, 3, 0, ..., 3, 0, 3])

In [8]:
# same probability for all the instances
constant_probability = np.full_like(y_test, 1/7)

y_pred_baseline = np.argmax(constant_probability, axis=1)

# Calculate metrics
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_baseline)
precision = precision_score(np.argmax(y_test, axis=1), y_pred_baseline, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred_baseline, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred_baseline, average='weighted')
auc = roc_auc_score(y_test, constant_probability, average='weighted')


/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
results_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC'],
    'value': [accuracy, precision, recall, f1, auc]
})

In [10]:
results_df

,Metric,value
0,Accuracy,0.289995
1,Precision,0.084097
2,Recall,0.289995
3,F1-score,0.130384
4,AUC,0.500000


## `Baseline` CNN

### CATEGORICAL MODEL

In [12]:
##next commente cell should be uncommented once
#labels_unique.Image_Folder=labels_unique.Image_Folder+"/naip.png"
##separate the datasets in train and test
train = int(labels_unique.shape[0] * 0.8)
test = int(labels_unique.shape[0] * 0.01)
labels_unique_train = labels_unique.iloc[:train, :]
labels_unique_test = labels_unique.iloc[-test:, :]

/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
labels_unique

,id,Latitude,Longitude,Type,Source,Image_Folder,geometry
0,1,43.004434,-78.208900,Negative,Aeroplanes,train_images/43.00443446102501_-78.20890003579...,"POLYGON ((-78.20567 43.00681, -78.20567 43.002..."
1,6,42.033642,-97.719996,Negative,"White Spec (Seadoo, pivot irrigation, etc.)",train_images/42.03364226519898_-97.71999550638...,"POLYGON ((-97.71676 42.03605, -97.71676 42.031..."
2,9,34.495000,-97.118000,WWTreatment,HydroWASTE,train_images/34.495_-97.118/naip.png,"POLYGON ((-97.11477 34.49768, -97.11477 34.492..."
3,12,38.381380,-75.401000,CAFOs,DataForCauseChallenge,train_images/38.38138_-75.401/naip.png,"POLYGON ((-75.39777 38.38393, -75.39777 38.378..."
4,17,45.647973,-84.469159,RefineriesAndTerminals,EIA-HIFLD,train_images/45.647973_-84.469159/naip.png,"POLYGON ((-84.46593 45.65024, -84.46593 45.645..."
...,...,...,...,...,...,...,...
14182,84941,35.852756,-101.538265,Negative,Windmills,train_images/35.85275555452138_-101.5382646083...,"POLYGON ((-101.53503 35.85539, -101.53503 35.8..."
14183,85008,47.186826,-122.436115,Negative,Roundabout,train_images/47.18682636726484_-122.4361145478...,"POLYGON ((-122.43288 47.18903, -122.43288 47.1..."
14184,85035,42.703517,-83.303270,Negative,Parking Lots,train_images/42.703517482995565_-83.3032698788...,"POLYGON ((-83.30004 42.70590, -83.30004 42.701..."
14185,85048,34.159700,-86.468760,CAFOs,DataForCauseChallenge,train_images/34.1597_-86.46876/naip.png,"POLYGON ((-86.46553 34.16239, -86.46553 34.157..."


In [14]:
##input data 
image_size = (720, 720)
batch_size = 64
validation_split = 0.2
test_split = 0.1

##get images from path on train and test 

from keras.preprocessing import image
dir_path=r"/root/code/dvictoria2/methane_source/"
train_datagen=image.ImageDataGenerator(rescale=1./255, validation_split = validation_split)
test_datagen =image.ImageDataGenerator(rescale=1./255)

In [15]:
##create generators for train_validation and test
train_generator=train_datagen.flow_from_dataframe(
    dataframe=labels_unique_train, directory=dir_path, 
    x_col="Image_Folder", y_col="Type", seed = 42,
    class_mode="categorical", target_size=(720,720), batch_size=batch_size , subset = "training", color_mode='rgb')
validation_generator = train_datagen.flow_from_dataframe(dataframe=labels_unique_train, directory=dir_path, 
    x_col="Image_Folder", y_col="Type", seed = 42,
    class_mode="categorical", target_size=(720,720), batch_size=int(batch_size/2), subset = "validation", color_mode='rgb')
test_generator = test_datagen.flow_from_dataframe(dataframe=labels_unique_test, directory=dir_path, 
    x_col="Image_Folder", y_col="Type", seed = 42,
    class_mode="categorical", target_size=(720,720), batch_size=141, color_mode='rgb')

Found 9040 validated image filenames belonging to 7 classes.
Found 2260 validated image filenames belonging to 7 classes.
Found 141 validated image filenames belonging to 7 classes.


In [16]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [49]:
##create build_model funtion
def build_model():
    inputs = layers.Input(shape=(720, 720, 3))
    res= layers.Resizing(360,360, interpolation='bilinear',
                         crop_to_aspect_ratio=False)(inputs)
    cnn = layers.Conv2D(32, (3, 3), activation='relu')(res)
    cnn2 = layers.MaxPooling2D((2, 2))(cnn)
    cnn3 = layers.Conv2D(64, (3, 3), activation='relu')(cnn2)
    cnn4 = layers.MaxPooling2D((2, 2))(cnn3)
    
    flatten = layers.Flatten()(cnn4)
    dense1 = layers.Dense(16, activation='relu')(flatten)
    out = layers.Dense(7, activation='softmax')(dense1)
    
    model = Model(inputs=inputs, outputs=out)
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        'accuracy'
    ])
    return model


In [50]:
model=build_model()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 720, 720, 3)]     0         
                                                                 
 resizing_2 (Resizing)       (None, 360, 360, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 358, 358, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 179, 179, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 177, 177, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 88, 88, 64)       0         
 2D)                                                       

In [51]:
cb = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(patience=3),callbacks.ModelCheckpoint('methane_m1.hdf5', monitor='loss',verbose=1, save_best_only=True)]
epochs = 20
# fits the model on batches with real-time data augmentation:
history =model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks = cb)



/tmp/ipykernel_10544/505452867.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history =model.fit_generator(generator=train_generator,


Epoch 1/10


2023-05-24 07:21:26.028125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-24 07:21:26.120039: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:21:26.120078: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:21:26.120092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-24 07:21:26.120102: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

141/141 [==============================] - ETA: 0s - loss: 1.7370 - auc: 0.8162 - pr_auc: 0.4393 - precision: 0.6119 - recall: 0.1800 - accuracy: 0.4508

2023-05-24 07:23:31.378643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-24 07:23:31.445828: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:23:31.445873: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:23:31.445901: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-24 07:23:31.445910: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f


Epoch 1: loss improved from inf to 1.73701, saving model to methane_m1.hdf5
141/141 [==============================] - 157s 1s/step - loss: 1.7370 - auc: 0.8162 - pr_auc: 0.4393 - precision: 0.6119 - recall: 0.1800 - accuracy: 0.4508 - val_loss: 1.1941 - val_auc: 0.8854 - val_pr_auc: 0.6381 - val_precision: 0.8591 - val_recall: 0.2777 - val_accuracy: 0.5455 - lr: 0.0010
Epoch 2/10
141/141 [==============================] - ETA: 0s - loss: 1.1256 - auc: 0.8930 - pr_auc: 0.6641 - precision: 0.7658 - recall: 0.3919 - accuracy: 0.5869
Epoch 2: loss improved from 1.73701 to 1.12558, saving model to methane_m1.hdf5
141/141 [==============================] - 148s 1s/step - loss: 1.1256 - auc: 0.8930 - pr_auc: 0.6641 - precision: 0.7658 - recall: 0.3919 - accuracy: 0.5869 - val_loss: 1.1069 - val_auc: 0.8974 - val_pr_auc: 0.6674 - val_precision: 0.7771 - val_recall: 0.3643 - val_accuracy: 0.5710 - lr: 0.0010
Epoch 3/10
141/141 [==============================] - ETA: 0s - loss: 1.0108 - auc: 0

In [53]:
model_name = "model1.pkl"
with open(model_name, 'wb') as archivo:
    pickle.dump(model, archivo)

INFO:tensorflow:Assets written to: ram://22c34352-e666-433f-bc9b-ccc548c9a303/assets


INFO:tensorflow:Assets written to: ram://22c34352-e666-433f-bc9b-ccc548c9a303/assets


In [18]:
import matplotlib.pyplot as plt

def plot_metric(metric_name, train_values, val_values, title):
    epochs = range(1, len(train_values) + 1)
    plt.plot(epochs, train_values, 'b', label='Train')
    plt.plot(epochs, val_values, 'r', label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.legend()
    plt.show()

def plot_metrics(history):
    # metrics
    plot_metric('AUC', history.history['auc'], history.history['val_auc'], 'AUC - Training and Validation')
    plot_metric('Precision', history.history['precision'], history.history['val_precision'], 'Precision - Training and Validation')
    plot_metric('Recall', history.history['recall'], history.history['val_recall'], 'Recall - Training and Validation')
    
    # loss
    plot_metric('Loss', history.history['loss'], history.history['val_loss'], 'Loss - Training and Validation')




In [ ]:
plot_metrics(history)


In [56]:
# evaluate the test set
loss, auc, pr_auc, precision, recall, accuracy =model.evaluate(test_generator, steps=STEP_SIZE_TEST)

44/44 [==============================] - 38s 858ms/step - loss: 1.5775 - auc: 0.8714 - pr_auc: 0.6649 - precision: 0.6732 - recall: 0.5941 - accuracy: 0.6392


In [57]:
resu= pd.DataFrame({'metrics': ['loss', 'auc', 'pr_auc',' precision', 'recall', 'accuracy'],
                    'value': [loss, auc, pr_auc, precision, recall, accuracy]})

#### OVERALL RESULTS FOR CATEGORICAL MODEL

In [58]:
resu

,metrics,value
0,loss,1.577510
1,auc,0.871361
2,pr_auc,0.664941
3,precision,0.673239
4,recall,0.594105
5,accuracy,0.639205


### BINARY MODEL 

In [59]:
def build_model2():
    inputs = layers.Input(shape=(720, 720, 3))
    res= layers.Resizing(360,360, interpolation='bilinear',
                         crop_to_aspect_ratio=False)(inputs)
    cnn = layers.Conv2D(32, (3, 3), activation='relu')(res)
    cnn2 = layers.MaxPooling2D((2, 2))(cnn)
    cnn3 = layers.Conv2D(64, (3, 3), activation='relu')(cnn2)
    cnn4 = layers.MaxPooling2D((2, 2))(cnn3)
    
    flatten = layers.Flatten()(cnn4)
    dense1 = layers.Dense(16, activation='relu')(flatten)
    out = layers.Dense(7, activation='sigmoid')(dense1)
    
    model = Model(inputs=inputs, outputs=out)
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        'accuracy'
    ])
    return model

In [60]:
model2=build_model2()
model2.summary()
cb = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(patience=3),callbacks.ModelCheckpoint('methane_m2.hdf5', monitor='loss',verbose=1, save_best_only=True)]
epochs = 20
# fits the model on batches with real-time data augmentation:
history2 =model2.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks = cb)

loss2, auc2, pr_auc2, precision2, recall2, accuracy2 =model2.evaluate(test_generator, steps=STEP_SIZE_TEST)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 720, 720, 3)]     0         
                                                                 
 resizing_3 (Resizing)       (None, 360, 360, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 358, 358, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 179, 179, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 177, 177, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 88, 88, 64)       0         
 2D)                                                       

/tmp/ipykernel_10544/3318589599.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history2 =model2.fit_generator(generator=train_generator,


Epoch 1/10


2023-05-24 07:46:52.791055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-24 07:46:52.889881: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:46:52.889923: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:46:52.889939: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-24 07:46:52.889950: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

141/141 [==============================] - ETA: 0s - loss: 0.5301 - auc: 0.6840 - pr_auc: 0.2352 - precision: 0.2306 - recall: 0.3680 - accuracy: 0.2894

2023-05-24 07:48:50.895385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-24 07:48:50.964222: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:48:50.964289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-24 07:48:50.964307: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-24 07:48:50.964318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f


Epoch 1: loss improved from inf to 0.53012, saving model to methane_m2.hdf5
141/141 [==============================] - 149s 1s/step - loss: 0.5301 - auc: 0.6840 - pr_auc: 0.2352 - precision: 0.2306 - recall: 0.3680 - accuracy: 0.2894 - val_loss: 0.3280 - val_auc: 0.8450 - val_pr_auc: 0.5651 - val_precision: 0.7066 - val_recall: 0.3000 - val_accuracy: 0.5147 - lr: 0.0010
Epoch 2/10
141/141 [==============================] - ETA: 0s - loss: 0.2960 - auc: 0.8606 - pr_auc: 0.6106 - precision: 0.7418 - recall: 0.3507 - accuracy: 0.5511
Epoch 2: loss improved from 0.53012 to 0.29602, saving model to methane_m2.hdf5
141/141 [==============================] - 151s 1s/step - loss: 0.2960 - auc: 0.8606 - pr_auc: 0.6106 - precision: 0.7418 - recall: 0.3507 - accuracy: 0.5511 - val_loss: 0.2892 - val_auc: 0.8559 - val_pr_auc: 0.5991 - val_precision: 0.7774 - val_recall: 0.3040 - val_accuracy: 0.5165 - lr: 0.0010
Epoch 3/10
141/141 [==============================] - ETA: 0s - loss: 0.2682 - auc: 0

In [61]:
model_name = "model2.pkl"
with open(model_name, 'wb') as archivo:
    pickle.dump(model2, archivo)

INFO:tensorflow:Assets written to: ram://15c15429-a133-4612-be0a-8771a349f054/assets


INFO:tensorflow:Assets written to: ram://15c15429-a133-4612-be0a-8771a349f054/assets


In [21]:
model_name = "model2.pkl"
with open(model_name, 'rb') as archivo:
    model2 = pickle.load(archivo)

2023-05-25 00:45:48.902361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 00:45:48.903332: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (NVIDIA RTX A4000 Laptop GPU)
2023-05-25 00:45:51.004943: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 1 (Intel(R) UHD Graphics)
2023-05-25 00:45:51.023251: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 00:45:51.023270: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU 

OVERALL RESULTS FOR BINARY MODEL

In [22]:
plot_metrics(history2)

NameError: name 'history2' is not defined

In [63]:
resu2= pd.DataFrame({'metrics': ['loss', 'auc', 'pr_auc',' precision', 'recall', 'accuracy'],
                    'value': [loss2, auc2, pr_auc2, precision2, recall2, accuracy2]})

In [24]:
X_test, y_test = test_generator.next()

In [30]:
class_indices = train_generator.class_indices
category_names = {v: k for k, v in class_indices.items()}
print(category_names)

{0: 'CAFOs', 1: 'Landfills', 2: 'Mines', 3: 'Negative', 4: 'ProcessingPlants', 5: 'RefineriesAndTerminals', 6: 'WWTreatment'}


In [25]:
y_pred_binary=model2.predict(X_test)

2023-05-25 00:46:25.373198: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 2s 84ms/step


2023-05-25 00:46:27.356555: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 00:46:27.356594: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 00:46:27.356626: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-25 00:46:27.356637: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:1 with 45099 MB memory) -> physical PluggableDevice (device: 1, name: 

In [26]:
y_pred_binary

array([[6.09411811e-03, 2.18743198e-02, 2.19237767e-02, 6.84730828e-01,
        7.62199331e-03, 9.75158811e-03, 1.01991169e-01],
       [6.29239455e-02, 5.44129126e-02, 1.99048554e-05, 4.36632335e-02,
        1.27374951e-04, 3.13976780e-04, 2.48269871e-01],
       [5.82828606e-03, 6.92123994e-02, 5.82708628e-04, 5.66777945e-01,
        2.69539480e-04, 1.80881715e-03, 9.16389674e-02],
       [1.18569827e-04, 8.23305994e-02, 1.36085097e-02, 2.82099247e-01,
        2.74655665e-03, 7.07901223e-03, 1.66381802e-02],
       [9.97964144e-01, 1.94839144e-04, 1.53004803e-05, 1.34610484e-04,
        8.56825645e-05, 1.85248386e-02, 1.23913391e-02],
       [1.67387892e-09, 9.64232605e-09, 3.69325280e-05, 9.72581387e-01,
        6.30556727e-08, 5.38636300e-07, 1.29290620e-05],
       [9.91082370e-01, 3.59460137e-05, 9.24909909e-06, 1.27177322e-02,
        2.62927533e-05, 1.60332012e-03, 4.44960967e-02],
       [2.14899238e-03, 4.29880247e-03, 2.61795678e-04, 9.84019518e-01,
        3.73827759e-03, 2

In [27]:
binarized_labels = np.argmax(y_test, axis=1)
binarized_predictions = np.argmax(y_pred_binary, axis=1)

precisions = []
recalls = []
auc_values = []
pr_auc_values=[]
accuracy_values=[]


num_categories = y_test.shape[1]
for category in range(num_categories):
    labels_category = (binarized_labels == category).astype(int)
    predictions_category = (binarized_predictions == category).astype(int)
    
    precision = tf.keras.metrics.Precision(name='precision')(labels_category, predictions_category)
    recall = tf.keras.metrics.Recall(name='recall')(labels_category, predictions_category)
    auc = tf.keras.metrics.AUC(name='auc')(labels_category, y_pred_binary[:, category])
    pr_auc=tf.keras.metrics.AUC(name='pr_auc', curve='PR')(binarized_labels, binarized_predictions)
    accuracy=tf.keras.metrics.Accuracy(name='accuracy')(binarized_labels, binarized_predictions)
    
    precisions.append(precision.numpy())
    recalls.append(recall.numpy())
    auc_values.append(auc.numpy())
    pr_auc_values.append(pr_auc.numpy())
    accuracy_values.append(accuracy.numpy())

2023-05-25 00:46:44.631269: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 00:46:44.631324: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 00:46:44.631351: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-25 00:46:44.631368: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:1 with 45099 MB memory) -> physical PluggableDevice (device: 1, name: 

## baseline metrics for CNN

In [37]:
category_names = {v: k for k, v in class_indices.items()}

metrics_df = pd.DataFrame({
    'Categoría': [category_names[i] for i in range(num_categories)],
    'Precision': precisions,
    'Recall': recalls,
    'AUC': auc_values,
    'AUC_PR': pr_auc_values,
    'Accuracy': accuracy_values
})

print(metrics_df)

                Categoría  Precision    Recall       AUC    AUC_PR  Accuracy
0                   CAFOs   0.571429  0.600000  0.811157  0.930387  0.673759
1               Landfills   0.000000  0.000000  0.645588  0.930387  0.673759
2                   Mines   0.000000  0.000000  0.585749  0.930387  0.673759
3                Negative   0.830000  0.864583  0.845139  0.930387  0.673759
4        ProcessingPlants   0.000000  0.000000  0.500912  0.930387  0.673759
5  RefineriesAndTerminals   0.000000  0.000000  0.529393  0.930387  0.673759
6             WWTreatment   0.000000  0.000000  0.767857  0.930387  0.673759


## this model 3 is an attemp of 6 categories, still on working 

In [32]:
def build_model3():
    inputs = layers.Input(shape=(720, 720, 3))
    res= layers.Resizing(360,360, interpolation='bilinear',
                         crop_to_aspect_ratio=False)(inputs)
    cnn = layers.Conv2D(32, (3, 3), activation='relu')(res)
    cnn2 = layers.MaxPooling2D((2, 2))(cnn)
    cnn3 = layers.Conv2D(64, (3, 3), activation='relu')(cnn2)
    cnn4 = layers.MaxPooling2D((2, 2))(cnn3)
    
    flatten = layers.Flatten()(cnn4)
    dense1 = layers.Dense(16, activation='relu')(flatten)
    out = layers.Dense(6, activation='sigmoid')(dense1)
    
    model = Model(inputs=inputs, outputs=out)
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        'accuracy'
    ])
    return model

In [33]:
model3=build_model3()
model3.summary()
cb = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(patience=3),callbacks.ModelCheckpoint('methane_m2.hdf5', monitor='loss',verbose=1, save_best_only=True)]
epochs = 20
# fits the model on batches with real-time data augmentation:
history3 =model3.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks = cb)

loss3, auc3, pr_auc3, precision3, recall3, accuracy3 =model3.evaluate(test_generator, steps=STEP_SIZE_TEST)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 720, 720, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 360, 360, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 358, 358, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 179, 179, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 177, 177, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 88, 88, 64)       0         
 2D)                                                         

/tmp/ipykernel_1147/3508403701.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history3 =model3.fit_generator(generator=train_generator,


Epoch 1/10


2023-05-25 01:01:57.156770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-25 01:01:57.297311: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 01:01:57.297404: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 01:01:57.297482: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 51939 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-25 01:01:57.297497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

InvalidArgumentError: Graph execution error:

Detected at node 'binary_crossentropy/logistic_loss/mul' defined at (most recent call last):
    File "/root/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/root/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
 